<h1 align="center">Jingyang`s Final Assignment</h1>

<p style='text-align: justify;'> 
Hi, glad to meet you at here. I wrote those words/codes to get a training in Python data science. 

This notebook contains the raw data, with my analysis, discusssion and conclusion. So, it would be also used as the thesis for the assignment.

</p>

## Part 1 Project Design  

<p style='text-align: justify;'> 
I want to explore the relationship between the venues trending and their positions. It is quite common to think that those hot venues may be next to each other. But the thing is, how close they are and if it is possible for us to find the possibilities of creating a new "hot group". That`s what I want to do with FourSquare.

My protential audience would be self-employed persons and investors. For self-employers like shop owners, they can make a good use of my result to have a better understand of the trend of business district, they can wisely choose where to do there business. For investors, they can also have a better understanding of the business trends, but from higher perspectives. They can use my results to create new business distince more efficiently, which is essential for them to make more money. 

I will be using the data from FourSquare and some geo-related Python libraries. I plan to use the data from the FourSquare about the trending and position of venues and with the help of Python packages to do visualization and machine learning. 

For example, I can get the check-in number of people of each venue, which would be an important factor that determines whether it is a hot place or not. Then, I will calculate the number limit for determining "hot" or not. I plan to choose the top 20% venues. Then I will get their physical position via Python librarys. Then do the visualization and clustering. 
</p>

## Part 2 Data Analysis Work

<p style='text-align: justify;'> 
I will work on the data of Downtown Toronto since I have already done some exploration on Toronto. Base on the geographic data of downtown Toronto, FourSquare API would be applied to get the information of venues nearby. 
</p>

In [2]:
import pandas as pd
import numpy as np

In [19]:
data_table = pd.read_html(r"https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

In [31]:
data = data_table[1:]
data.columns = data_table.iloc[0]
data = data[data["Borough"] != "Not assigned"].reset_index(drop = True)

In [34]:
data["Neighborhood"] = data["Neighborhood"].str.replace(" /", ",")

In [37]:
data.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [35]:
geo_posi = pd.read_csv("Geospatial_Coordinates.csv")

In [39]:
geo_posi.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [46]:
ans_df = pd.merge(data, geo_posi, how = 'left', left_on = "Postal code", right_on = "Postal Code").drop(columns = "Postal Code")
ans_df.columns = ["Postal Code", "Borough", "Neighborhood", "Latitude", "Longitude"]
ans_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [47]:
ans_df.to_csv("Borough_With_Position.csv")

Now I have the geographic data of Toronto. The \"Downtown Toronto\" would be extracted for further analysis.

### Preparative Data Visualization

<p style='text-align: justify;'> First, I need to have a look at downtown toronto on the map. If it does not have a high density of people, then it would be meaningless for me to do the analysis. The density of people would be represented by the density of postal code since they are always positively related.
</p>

In [3]:
r_posi_df = pd.read_csv("Borough_With_Position.csv")

In [4]:
posi_df = r_posi_df[["Postal Code", "Borough", "Neighborhood", "Latitude", "Longitude"]]
posi_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [5]:
posi_df.groupby('Borough').count().sort_values(by = ['Postal Code'],
                                               ascending=False).head()

,Postal Code,Neighborhood,Latitude,Longitude
Borough,,,,
North York,24,24,24,24
Downtown Toronto,19,19,19,19
Scarborough,17,17,17,17
Etobicoke,12,12,12,12
Central Toronto,9,9,9,9


<p style='text-align: justify;'> So, it looks like North York and Downtown Toronto have the most postal codes, let`s try to do the visualization.
</p>

In [6]:
densi_Bor_df = posi_df.loc[posi_df['Borough'].isin(['North York',
                                               'Downtown Toronto'])]

densi_Bor_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [7]:
from geopy.geocoders import Nominatim
import folium # map rendering library

In [8]:
address = "Toronto"

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'
      .format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [9]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough in zip(densi_Bor_df['Latitude'],
                                           densi_Bor_df['Longitude'],
                                           densi_Bor_df['Borough']):
    if borough == 'North York':
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_toronto)
    else:
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            color='red',
            fill=True,
            fill_color='#cc315a',
            fill_opacity=0.7,
            parse_html=False).add_to(map_toronto)
    
map_toronto

<p style='text-align: justify;'> From the map, we can see that the reason why North York has many postal codes is it is a big area, not because of high density. The Downtown Toronto has a bigger code density, which is consistent with how we think of "Downtown". So I am going to give a further analysis to the downtown area only. 
</p>

### Downtown Analysis

In [10]:
dt_Toronto = densi_Bor_df[densi_Bor_df['Borough'] == 'Downtown Toronto'].reset_index(drop = True)

dt_Toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [11]:
address = "Downtown Toronto"

geolocator = Nominatim(user_agent="my_explorer")
dt_location = geolocator.geocode(address)
dt_latitude = dt_location.latitude
dt_longitude = dt_location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'
      .format(dt_latitude, dt_longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


In [12]:
map_Downtown_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng in zip(dt_Toronto['Latitude'], dt_Toronto['Longitude']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Downtown_toronto)
    
map_Downtown_toronto

<p style='text-align: justify;'> Now let me find the density distribution of this area. It is hard to say how to seperate "Donwntown Toronto", so density might give us a good insight about the distribution.
</p>

In [13]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler 
import matplotlib.pyplot as plt 
%matplotlib inline

In [14]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

In [15]:
densi_clst = dt_Toronto[['Latitude', 'Longitude']]
densi_clst = StandardScaler().fit_transform(densi_clst)

db = DBSCAN(eps=0.4, min_samples=4).fit(densi_clst)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

dt_Toronto['Area'] = labels

In [16]:
set(labels)

{-1, 0}

In [17]:
db_map_Downtown_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, lab in zip(dt_Toronto['Latitude'],
                         dt_Toronto['Longitude'], 
                        dt_Toronto['Area']):
    if lab == -1:
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            color= 'blue',
            parse_html=False).add_to(db_map_Downtown_toronto)
    
    else:
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            color='red',
            parse_html=False).add_to(db_map_Downtown_toronto)

db_map_Downtown_toronto

<p style='text-align: justify;'> I have done some search for the most reasonable distance for cluster division. From the result of the visualization, the result above seems to be the most reasonable one. I am trying to find a "core" among the downtown area, and the red markers above are places that I want to further analyze.
</p>

In [18]:
# Seperate the CORE POINTS
core_dt_tto = dt_Toronto[dt_Toronto['Area'] == 0].reset_index(drop = True)
core_dt_tto

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Area
0,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0
1,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,0
2,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0
3,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,0


In [19]:
core_dt_tto.to_csv("Core_Down_Toronto.csv")

### FourSquare based Analysis

My personal information was removed for sharing

In [21]:
m5h_la, m5h_lo = core_dt_tto.loc[0, ['Latitude', 'Longitude']]
m5k_la, m5k_lo = core_dt_tto.loc[1, ['Latitude', 'Longitude']]
m5l_la, m5l_lo = core_dt_tto.loc[2, ['Latitude', 'Longitude']]
m5x_la, m5x_lo = core_dt_tto.loc[3, ['Latitude', 'Longitude']]

In [22]:
m5h_la

43.65057120000001

In [23]:
import json
import requests
from pandas.io.json import json_normalize

In [24]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
def acquire_venues(la, lo):
    LIMIT = 100
    radius = 1000
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    la, 
    lo, 
    radius, 
    LIMIT)
    
    results = requests.get(url).json()
    
    venues = results['response']['groups'][0]['items']
    
    nearby_venues = json_normalize(venues) # flatten JSON

    # filter columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    nearby_venues =nearby_venues.loc[:, filtered_columns]

    # filter the category for each row
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

    # clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
    
    return nearby_venues


In [26]:
m5h_places = acquire_venues(m5h_la, m5h_lo)
m5k_places = acquire_venues(m5k_la, m5k_lo)
m5l_places = acquire_venues(m5l_la, m5l_lo)
m5x_places = acquire_venues(m5x_la, m5x_lo)

In [27]:
m5h_places.head()

,name,categories,lat,lng
0,Nathan Phillips Square,Plaza,43.652270,-79.383516
1,Four Seasons Centre for the Performing Arts,Concert Hall,43.650592,-79.385806
2,Downtown Toronto,Neighborhood,43.653232,-79.385296
3,Soho House Toronto,Speakeasy,43.648734,-79.386541
4,Adelaide Club Toronto,Gym / Fitness Center,43.649279,-79.381921


In [28]:
m5k_places.head()

,name,categories,lat,lng
0,Canoe,Restaurant,43.647452,-79.381320
1,Mos Mos Coffee,Café,43.648159,-79.378745
2,Adelaide Club Toronto,Gym / Fitness Center,43.649279,-79.381921
3,Equinox Bay Street,Gym,43.648100,-79.379989
4,WVRST,Beer Bar,43.644968,-79.381376


In [29]:
def make_Map(posi_df):
    tgtMap = folium.Map(location=[43.6541737, -79.38081164513409], zoom_start=14)

    # add markers to map
    for lat, lng in zip(posi_df['lat'], posi_df['lng']):
        folium.CircleMarker(
            [lat, lng],
            radius=2,
            color='blue',
            parse_html=False).add_to(tgtMap)

    return tgtMap

In [91]:
make_Map(m5h_places)

In [31]:
make_Map(m5k_places)

From the two map above, I have realized that I have used a too big radium. So I decide to merge all four place together.

In [32]:
m5h_ll = m5h_places[['lat', 'lng']]
m5k_ll = m5k_places[['lat', 'lng']]
m5l_ll = m5l_places[['lat', 'lng']]
m5x_ll = m5x_places[['lat', 'lng']]

In [33]:
h_k = pd.merge(m5h_ll, m5k_ll, how = 'outer')
l_x = pd.merge(m5l_ll, m5x_ll, how = 'outer')

In [34]:
core_places = pd.merge(h_k, l_x, how = 'outer')
core_places.head()

,lat,lng
0,43.652270,-79.383516
1,43.650592,-79.385806
2,43.653232,-79.385296
3,43.648734,-79.386541
4,43.649279,-79.381921


In [41]:
core_center_position = core_places.mean()
core_center_position

lat    43.649018
lng   -79.382210
dtype: float64

In [37]:
make_Map(core_places)

### ML Analysis 

In [82]:
color_choices = list({'black', 'blue', 'gray', 'green', 'orange', 'pink', 'purple', 'red', 'white'})

def clst_map(eps, min_samples):
    coreplace_clst = core_places[['lat', 'lng']]
    coreplace_clst = StandardScaler().fit_transform(coreplace_clst)

    core_db = DBSCAN(eps, min_samples).fit(coreplace_clst)
    core_samples_mask = np.zeros_like(core_db.labels_, dtype=bool)
    core_samples_mask[core_db.core_sample_indices_] = True
    labels = core_db.labels_

    core_places['Group'] = labels
    core_place_map = folium.Map(location=[43.649018, -79.382210], 
                                tiles = 'Stamen Toner',
                                zoom_start=14.5)
    

    # add markers to map
    for lat, lng, grp in zip(core_places['lat'], core_places['lng'],  core_places['Group']):

        folium.CircleMarker(
            [lat, lng],
            radius=5,
            color= color_choices[grp+1],
            fill = True,
            parse_html=False).add_to(core_place_map)

    core_place_map.save('{},{}.html'.format(eps, min_samples))
    
    return core_place_map

In [84]:
clst_map(0.2,6)

In [87]:
clst_map(0.22,6)

In [88]:
clst_map(0.24,6)

In [93]:
clst_map(0.26,6)

In [90]:
clst_map(0.28,6)

In [92]:
clst_map(0.3,6)

## Part 3 Discussion

<p style='text-align: justify;'> 
Now, the analysis has been done. From the result of clustering, we can clearly find the tendency of the growth of business district in downtown Toronto. 

When we gradually reduce the clustering standard of the business district, we can find the direction their growth. More and more parts could be considered as districts. In some ways, the level of clustering could be considered as the level of business district. 

Overall, there was a "core" district, which existed only in the highest level requirement. And many related small districts. There was no doubt that the price of the center would be very high, however, there might also be more chance to make money.

Personally, I would suggested self-employed people to make their business near the core business area or, if the price at that place is too high, near the core area of other small business district. Compared with other places, those places are more potential.

</p>

## Part 4 Conclusion & Outlook

### Conclusion
<p style='text-align: justify;'> 
In this work, I have done the data analysis of the center of downtown Toronto. The business district of the downrown has been analyzed with FourSquare API. 

From the distribution of shops, restuarants and other places in the downtown area, it can be found that the distribution is not even but could be clustered into several business districts according to different levels.

Based on the result of analysis, suggestion for self-employed people could be given on how to choose a place to do business. Places near the center of business district would be good choices.
</p>

### Further Analysis
<p style='text-align: justify;'> 
In further research, I want to collect and analyze the data at different times. That can give us clearer picture about the change of downtown Toronto. With data based on time and distribution together, more helpful could be given for business people .


</p>